<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/MLBDataAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -q https://github.com/profteachkids/StemUnleashed/raw/main/mlb_batting.zip -O mlb_batting.zip

In [2]:
import numpy as np
import pandas as pd
import shutil
from pathlib import Path

In [3]:
shutil.unpack_archive('mlb_batting.zip',extract_dir='mlb_batting')

In [4]:
files=list(Path('mlb_batting').glob('**/*.csv'))
keys=[afile.as_posix().split('/')[1:] for afile in files]
keys=[tuple((*key[:-1],key[-1].split('.')[0])) for key in keys]

In [5]:
df_list=[]
days=['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
for afile in files:
    df= pd.read_csv(afile).iloc[:,1:]
    df['home']=~df['OPP'].str.startswith('@')
    df['win']=df['Result'].str.startswith('W')
    df=df.loc[df['Date'].str[:3].isin(days),['Date','home','AB','HR', 'win']]
    df[['day','Date']]=df.loc[:,'Date'].str.split(' ',expand=True).loc[:,:1]
    df_list.append(df)

In [6]:
df=pd.concat(df_list,keys=keys)
df.index=df.index.droplevel(3)
df.index.names=['team','player','year']
df=df.reset_index().set_index(['team','player'])
df[['AB','HR']]=df[['AB','HR']].astype(int)
df['day']=df['day'].astype('category')

In [7]:
#combine month/day with year
df['date']=pd.to_datetime(df['Date']+'/' +df['year'])
df=df.drop(['Date','year'],axis=1)

In [8]:
df_team=df.pivot_table(values=['AB','HR'],index=['team','home'],aggfunc=['sum'])

In [9]:
df_team.columns=df_team.columns.droplevel(0)

In [10]:
df_team['HR_percentage']=df_team['HR']/df_team['AB']

In [11]:
df_ratio=pd.pivot_table(df_team, index='team',columns='home',values='HR_percentage')

In [12]:
df_ratio

home,False,True
team,,
arizona-diamondbacks,0.034471,0.030386
atlanta-braves,0.043194,0.043789
baltimore-orioles,0.035093,0.041690
boston-red-sox,0.035879,0.040978
chicago-cubs,0.030992,0.032380
chicago-white-sox,0.032907,0.039290
cincinnati-reds,0.035382,0.039470
cleveland-guardians,0.034495,0.032419
colorado-rockies,0.033055,0.042727
